## Import packages

In [48]:
import Bio 
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

print(os.getcwd())


/Users/henrysun_1/Desktop/Duke/PhD 2025-2026/callinectes_co1


## Create reverse complement of old sequences

In [27]:
input_file = "raw/Megan.nexus"
output_file = "raw/Megan_revcomp.nex"
revcomp_records = []

for seq_record in SeqIO.parse(input_file, "nexus"):
    reverse_complement_seq = seq_record.seq.reverse_complement()
    reverse_complement_record = SeqRecord(
        reverse_complement_seq,
        id=seq_record.id,
        description=seq_record.description + " reverse_complemented",
    )
    reverse_complement_record.annotations["molecule_type"] = "DNA"
    revcomp_records.append(reverse_complement_record)

# Write all at once
with open(output_file, "w") as output_handle:
    SeqIO.write(revcomp_records, output_handle, "nexus")


## Join all sequences together and trim to overlapping region

In [12]:
# List your input NEXUS files
input_files = ["raw/Sardinia.nexus", "raw/Megan_revcomp.nex", "raw/Rasit_revcomp.nex"]
# input_files = ["raw/Rasit.nex", "raw/Sardinia.nexus"]

# Output FASTA file
output_file = "processed/combined_CO1.fasta"

# Collect all sequences
all_records = []
for f in input_files:
    for record in SeqIO.parse(f, "nexus"):
        all_records.append(record)

# Write all sequences (unaligned) to one FASTA
SeqIO.write(all_records, output_file, "fasta")


316

sites 106-588 common between new and old sequences  
save aligned file using mega

In [ ]:
## attempting to trim out excess
input_file = "processed/combined_CO1_aligned.fas"
output_file = "processed/combined_CO1_trimmed.fasta"

start = 106  # 1-based inclusive
end = 588    # 1-based inclusive

trimmed_records = []

for record in SeqIO.parse(input_file, "fasta"):
    trimmed_seq = record.seq[start-1:end]
    record.seq = trimmed_seq
    trimmed_records.append(record)
    
SeqIO.write(trimmed_records, output_file, "fasta")

316

In [ ]:
## TODO 11/10:
'''
extract FASTA file header names for raw / combined 
    - remove Mattamuskeet duplicates, check that sequences are identical
    - check for any other duplicates in combined file
'''

### Extract FASTA headers and remake FASTA with unique sequences


In [16]:
input_file = ["raw/Rasit_revcomp.nex"]
output_file = "raw/Rasit_revcomp.fasta"
rasit_records = []
for f in input_file:
    for record in SeqIO.parse(f, "nexus"):
        rasit_records.append(record)

# Write all sequences (unaligned) to one FASTA
SeqIO.write(rasit_records, output_file, "fasta")


162

In [ ]:
def get_fasta_headers(fasta_file_path):
    headers = []
    with open(fasta_file_path, "r") as f:
        for record in SeqIO.parse(f, "fasta"):
            headers.append(record.description)
    return headers

rasit = "raw/Rasit_revcomp.fasta" 
megan = "raw/Megan_revcomp.fasta"
rasit_headers = get_fasta_headers(rasit)
megan_headers = get_fasta_headers(megan)

rasit_headers

In [40]:
def get_mt_headers(fasta_file_path, clean_underscores=False):
    """Return only headers starting with 'Mt'"""
    headers = [h for h in get_fasta_headers(fasta_file_path) if h.startswith("Mt")]
    if clean_underscores:
        headers = [h.replace("_", "") for h in headers]
    return headers

rasit_mt_headers = set(get_mt_headers(rasit))
megan_mt_headers = set(get_mt_headers(megan, clean_underscores=True))

mt_unique_to_rasit = rasit_mt_headers - megan_mt_headers
mt_unique_to_megan = megan_mt_headers - rasit_mt_headers
shared_mt = rasit_mt_headers & megan_mt_headers

print("Headers starting with 'Mt' unique to 'all':", mt_unique_to_rasit)
print("Headers starting with 'Mt' unique to 'megan':", mt_unique_to_megan)
print("Shared 'Mt' headers:", shared_mt)


Headers starting with 'Mt' unique to 'all': set()
Headers starting with 'Mt' unique to 'megan': set()
Shared 'Mt' headers: {'MtA1', 'MtC1', 'MtB3', 'MtC4', 'MtC5f', 'MtB4', 'MtC7', 'MtA3', 'MtA4', 'MtA5', 'MtC9f', 'MtB9', 'MtC8f', 'MtC6', 'MtB1', 'MtA9', 'MtB7', 'MtC4f', 'MtC7f', 'MtD1f', 'MtC5', 'MtB5', 'MtC6f', 'MtA7', 'MtC3', 'MtD2'}


In [36]:
def get_bft_headers(fasta_file_path, clean_underscores=True):
    """Return only headers starting with 'Mt'."""
    headers = [h for h in get_fasta_headers(fasta_file_path) if h.startswith("Bft")]
    if clean_underscores:
        headers = [h.replace("_", "") for h in headers]
    return headers

rasit = "raw/Rasit_revcomp.fasta" 
megan = "raw/Megan_revcomp.fasta"

rasit_bft_headers = set(get_bft_headers(rasit))
megan_bft_headers = set(get_bft_headers(megan))

bft_unique_to_rasit = rasit_bft_headers - megan_bft_headers
bft_unique_to_megan = megan_bft_headers - rasit_bft_headers
shared_bft = rasit_bft_headers & megan_bft_headers

print("Headers starting with 'Bft' unique to 'rasit':", bft_unique_to_rasit)
print("Headers starting with 'Bft' unique to 'megan':", bft_unique_to_megan)
print("Shared 'Bft' headers:", shared_bft)


Headers starting with 'Bft' unique to 'rasit': set()
Headers starting with 'Bft' unique to 'megan': {'Bft101', 'Bft121'}
Shared 'Bft' headers: {'Bft15', 'Bft10', 'Bft34', 'Bft17', 'Bft28', 'Bft4', 'Bft33', 'Bft30', 'Bft1', 'Bft5', 'Bft21', 'Bft11', 'Bft26', 'Bft12', 'Bft7', 'Bft3', 'Bft25', 'Bft8', 'Bft2', 'Bft6', 'Bft13', 'Bft27', 'Bft29', 'Bft9', 'Bft14', 'Bft20'}


In [56]:
rasit_file = "raw/Rasit_revcomp.fasta"
megan_file = "raw/Megan_revcomp.fasta"
sardinia_file = "raw/Sardinia.nexus"
rasit_unique_file = "raw/Rasit_unique.fasta"
combined_unique_file = "processed/combined_unique.fasta"

shared_headers = shared_mt | shared_bft
rasit_records = list(SeqIO.parse(rasit_file, "fasta"))
rasit_filtered = []

for rec in rasit_records:
    clean_header = rec.description.replace("_", "")
    if clean_header not in shared_headers:
        rasit_filtered.append(rec)

print(f"Original Rasit sequences: {len(rasit_records)}")
print(f"Filtered Rasit sequences: {len(rasit_filtered)} (removed {len(rasit_records) - len(rasit_filtered)})")

combined_records = []
combined_records.extend(list(SeqIO.parse(rasit_unique_file, "fasta")))
combined_records.extend(list(SeqIO.parse(megan_file, "fasta")))
combined_records.extend(list(SeqIO.parse(sardinia_file, "nexus")))

SeqIO.write(combined_records, combined_unique_file, "fasta")



Original Rasit sequences: 162
Filtered Rasit sequences: 110 (removed 52)


264

In [ ]:
output_file = "processed/combined_unique.fasta"

pattern = re.compile(r'^(>C)(\d+)')
# also manually cf replace Bft, Sardinia with _

with open(combined_unique_file) as infile, open(output_file, "w") as outfile:
    for line in infile:
        outfile.write(pattern.sub(r'\1_\2', line))


In [ ]:
# TODO
# remake combined nexus with correct names, no 'copy' - remove Mt/Bft from Rasit fasta
# convert names into trait matrix for haplotype analysis
# make haplotype map TCS network, send to Rasit/Dan


## Write nexus from trimmed


In [3]:
input_file = "processed/combined_CO1_trimmed.fasta"
output_file = "processed/combined_CO1_trimmed.nex"

records = []
for record in SeqIO.parse(input_file, "fasta"):
    record.annotations["molecule_type"] = "DNA"
    records.append(record)

count = SeqIO.write(records, output_file, "nexus")

